<a href="https://colab.research.google.com/github/RGivisiez/TensorFlow-Templates/blob/main/TensorBoard_Embedding_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Imports

In [3]:
import os

### Model Definitions

In [4]:
folder_path = './logs1'

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

### Initialize Tensorboard

In [9]:
%load_ext tensorboard

In [10]:
%tensorboard --logdir $folder_path --host 0.0.0.0

### Auxiliary Function

In [8]:
import os
import tensorflow as tf
from tensorboard.plugins import projector

def Write_Embedding_File(weights, file_path, metadata_file_name):

  '''
     weights: Any variable that can be transformed in a tf.Variable. In this
              case, it will be the values in the embedding layer.

     file_path: Path where the embedding.ckpt will be saved.

     metadata_file_name: Metadata file name.
  '''

  # Save the weights we want to analyze as a variable. Note that the first
  # value represents any unknown word, which is not in the metadata, here
  # we will remove this value.

  weights = tf.Variable(weights)

  # Create a checkpoint from embedding, the filename and key are the
  # name of the tensor.
  checkpoint = tf.train.Checkpoint(embedding=weights)
  checkpoint.save(os.path.join(file_path, "embedding.ckpt"))

  # Set up config.
  config = projector.ProjectorConfig()
  embedding = config.embeddings.add()

  # The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
  embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
  embedding.metadata_path = metadata_file_name
  projector.visualize_embeddings(file_path, config)

  return

### Write Embedding Files

In [5]:
import numpy as np

In [6]:
import string
alphabet = list(string.ascii_lowercase)

In [7]:
import pandas as pd

#Write a file .tsv with the original values in the
#dataset.

df = pd.DataFrame(np.random.rand(len(alphabet), 3),
                  columns=['feature1', 'feature2', 'feature3'])

df['Names'] = alphabet[:df.shape[0]]
df.set_index('Names', inplace=True)

df.to_csv(folder_path + "/metadata.tsv", sep="\t")

df.tail()

,feature1,feature2,feature3
Names,,,
v,0.175440,0.493535,0.251726
w,0.289284,0.664754,0.787775
x,0.269358,0.373589,0.447067
y,0.592897,0.120832,0.774373
z,0.259286,0.238275,0.745418


In [11]:
#Write the embedding file.
Write_Embedding_File(np.random.rand(df.shape[0], 10), folder_path,
                     'metadata.tsv')